<a href="https://colab.research.google.com/github/praffuln/generative-ai/blob/master/rag/rag-1/rag_indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print('hello world!!')

hello world!!


# Indexing

# Multi-representation Indexing

**Enviornment**

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain-chroma langchain youtube-transcript-api pytube google-generativeai langchain_google_genai langchain_community


# Setup Keys For GOOGLE_API_KEY AND LANGSMITH

In [7]:
# Assigning value to variable
GOOGLE_API_KEY=''
LANGCHAIN_API_KEY = ''

# Imports os, google Gemini EMbedding and Chat Classes with FAISS vectorstores

In [8]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS



# setup langsmith for tracing

In [9]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

## Initialize LLM And Embedding

In [10]:
# LLM with function call
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

# Create embeddings using a Google Generative AI model
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")



### Fetch the documents

In [18]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs.extend(loader.load())

print(len(docs))
print('\nprinting metadata...\n')
for doc in docs:
  print(doc.metadata)



2

printing metadata...

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}
{'source': 'https://lilianweng.github.io/posts/2024-02-05-human-data-quality/', 'title': "Thinking about High-Quality Human Data | Lil'Log", 'description': '[Special thank you to Ian Kivlichan for many useful pointers (E.g. the 100+ year old Nature paper “Vox populi”) and nice feedback. \uf8ffüôè ]\nHigh-quality data is the fu

In [ ]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | llm
    | StrOutputParser()
)

summaries = chain.batch(docs, {"max_concurrency": 5})

len(summaries)
for summary in summaries:
  print(summary)
  print('\n\n\n\n\n\n\n')

#Store summary in vectorstore and original Docs in byteStore

In [25]:
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_chroma import Chroma

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=embeddings)

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Docs linked to summaries
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]


summary_docs

[Document(metadata={'doc_id': '9d570c6f-5a6f-4478-b18f-c8131a41d52b'}, page_content='This document explores the concept of LLM-powered autonomous agents, which are systems using large language models (LLMs) as their core controllers. It breaks down the key components of such agents: planning, memory, and tool use.\n\n**Planning** involves breaking down complex tasks into smaller subgoals and reflecting on past actions to improve future performance. Techniques like Chain of Thought (CoT), Tree of Thoughts, and ReAct are discussed for task decomposition and self-reflection.\n\n**Memory** encompasses short-term memory (in-context learning) and long-term memory (external vector stores). The document delves into the concept of Maximum Inner Product Search (MIPS) and various algorithms like LSH, ANNOY, HNSW, FAISS, and ScaNN for fast retrieval from vector stores.\n\n**Tool Use** equips LLMs with the ability to interact with external APIs and tools. Examples like MRKL, TALM, Toolformer, ChatG

In [26]:
# Add
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

## Retrive documents

In [30]:
### Get similarity search

query = "Memory in agents"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]


Document(metadata={'doc_id': 'c97f0a47-9843-44e2-af16-82ee4f19f65c'}, page_content='This document explores the concept of LLM-powered autonomous agents, which are systems using large language models (LLMs) as their core controllers. It breaks down the key components of such agents: planning, memory, and tool use.\n\n**Planning** involves breaking down complex tasks into smaller subgoals and reflecting on past actions to improve future performance. Techniques like Chain of Thought (CoT), Tree of Thoughts, and ReAct are discussed for task decomposition and self-reflection.\n\n**Memory** encompasses short-term memory (in-context learning) and long-term memory (external vector stores). The document delves into the concept of Maximum Inner Product Search (MIPS) and various algorithms like LSH, ANNOY, HNSW, FAISS, and ScaNN for fast retrieval from vector stores.\n\n**Tool Use** equips LLMs with the ability to interact with external APIs and tools. Examples like MRKL, TALM, Toolformer, ChatGP

In [29]:
### Get complete document

retrieved_docs = retriever.get_relevant_documents(query,n_results=1)
retrieved_docs[0].page_content[0:500]


"\n\n\n\n\n\nLLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n"

# RAPTOR